In [30]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

import pickle
import pandas as pd
import numpy as np

# Set random seed
np.random.seed(0)

# homes = pd.read_csv('C:/Research/Data/Activity/Extrasensory-activity-featurefile-all_1000.csv')
homes = pd.read_csv('C:/Research/Data/Activity/streamprocessor-like-data-25hz/featureFile.csv')

homes['is_train'] = np.random.uniform(0, 1, len(homes)) <= .50
df = homes

train, test = df[df['is_train']==True], df[df['is_train']==False]

print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

features = df.columns[:9]

print(features)

labelName = 'activityLabel'
# labelName = 'actualActivity'


y=train[labelName]
#y = pd.factorize(train[labelName])[0]

# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=2, random_state=0, n_estimators=100)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(train[features], y)

preds=clf.predict(test[features])
accuracy = accuracy_score(test[labelName], preds)

print(accuracy)

print(pd.crosstab(test[labelName], preds, rownames=['Actual labels'], colnames=['Predicted labels']))


# View a list of the features and their importance scores
print(list(zip(train[features], clf.feature_importances_)))


filename = 'activity_randomforest.model'
pickle.dump(clf, open(filename, 'wb'))

# load the model from disk
clf = pickle.load(open(filename, 'rb'))

preds=clf.predict(test[features])
accuracy = accuracy_score(test[labelName], preds)

print(accuracy)


# Create confusion matrix
print(pd.crosstab(test[labelName], preds, rownames=['Actual labels'], colnames=['Predicted labels']))

from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(confusion_matrix(test[labelName], preds))
print(cm)

print(classification_report(test[labelName], preds))
print(precision_recall_fscore_support(test[labelName], preds, average='weighted'))

sns.heatmap(cm)
plt.show()


Number of observations in the training data: 80341
Number of observations in the test data: 80582
Index(['mean', 'median', 'sd', 'sd_x', 'sd_y', 'sd_z', 'skew', 'kurt',
       'rateOfChange'],
      dtype='object')
0.837804968852
Predicted labels  HIGH    LOW  MODERATE  NO_ACT
Actual labels                                  
HIGH              3787    793       231     149
LOW                177  19320       815    4408
MODERATE           271   2889      2686     341
NO_ACT              22   2929        45   41719
[('mean', 0.096187667507750554), ('median', 0.082364166409655049), ('sd', 0.12300186493020697), ('sd_x', 0.12530415518749297), ('sd_y', 0.20882691941285728), ('sd_z', 0.16294741981645075), ('skew', 0.069984093695550836), ('kurt', 0.080948364019944735), ('rateOfChange', 0.050435349020090864)]
0.837804968852
Predicted labels  HIGH    LOW  MODERATE  NO_ACT
Actual labels                                  
HIGH              3787    793       231     149
LOW                177  19320 

NameError: name 'sns' is not defined

In [ ]:
# http://www.blopig.com/blog/2017/07/using-random-forests-in-python-with-scikit-learn/
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

import pickle
import pandas as pd
import numpy as np

def test_classifier(x_test):
    filename = 'activity_randomforest.model'
    # load the model from disk
    clf = pickle.load(open(filename, 'rb'))

    preds=clf.predict(x_test)
    
    return preds

if __name__ == "__main__":
        
    homes = pd.read_csv('C:/Research/Data/Activity/Extrasensory-activity-featurefile-all_1000.csv')
    homes['is_train'] = np.random.uniform(0, 1, len(homes)) <= .50
    df = homes

    train, test = df[df['is_train']==True], df[df['is_train']==False]

    print('Number of observations in the training data:', len(train))
    print('Number of observations in the test data:',len(test))

    features = df.columns[:9]

    print(features)

    labelName = 'activityLabel'
    # labelName = 'actualActivity'


    preds=test_classifier(test[features])
    accuracy = accuracy_score(test[labelName], preds)

    print(accuracy)

    print(pd.crosstab(test[labelName], preds, rownames=['Actual labels'], colnames=['Predicted labels']))



In [ ]:
import pandas as pd

df=pd.DataFrame({ 'x':range(30) })
df = df.rolling(10).mean()           # version 0.18.0 syntax
df[4::5] 
print(df)

In [8]:
import numpy as np, pandas as pd
import math
from scipy.stats import skew
from scipy.stats import kurtosis

def getRateOfChange(timestamp, magnitude):
    roc = 0;    
    for i in range(len(magnitude)-1):                
        roc = roc + (((magnitude[i+1] - magnitude[i]) / (timestamp.iloc[i+1] - timestamp.iloc[i])));
    roc = roc / (len(magnitude)- 1);
    
#     print("roc:",roc);
    
    return roc

def computeFeatures(start_time, end_time, time, x, y, z):
    
#     print("start_time:", start_time)
#     print("end_time:", end_time)
#     print("time:",time)
#     print("x:",x)
#     print("y:",y)
#     print("z:",z)
            
    
    mag = np.empty([len(x), 1])
    for i in range(len(x)):
        mag[i] = math.sqrt(x.iloc[i]*x.iloc[i] + y.iloc[i]*y.iloc[i]+z.iloc[i]*z.iloc[i])

    #a = [2, 8, 0, 4, 1, 9, 9, 0];
    mean =  np.mean(mag);
    median = np.median(mag);
    std = np.std(mag);
    std_x = np.std(x);
    std_y = np.std(y);
    std_z = np.std(z);
    skewness = skew(mag);
    kurt = kurtosis(mag);
    
    print("Mean:", mean);
    print("Meadian:",median);
    print("Std:", std);
    print("Std_x",std_x);
    print("Std_y", std_y);
    print("Std_z",std_z);
    print("Skewness:",skewness);
    print("Kurtosis:",kurt);
    
    rateOfChanges=getRateOfChange(time, mag)
    
    f = [start_time, end_time,mean, median, std, std_x, std_y, std_z, skewness, kurt, rateOfChanges]
#     print("f:", f)
    return f

if __name__ == "__main__":
    column_names = ['timestamp', 'A_x', 'A_y', 'A_z']
    D = pd.read_csv('C:/Research/Data/Activity/right-wrist-accelxyz.csv', names = column_names)
    
    x= D['A_x']
    y= D['A_y']
    z= D['A_z']
    time= D['timestamp']
    
    time_prev = time[0]
    prev = 0
    f_all = []
    for i in range(len(time)):
        if (time[i] - time_prev) >= 10000:
           # print(time[i])
           # print(time_prev)
            time_sub = time[prev:i+1]
            x_sub = x[prev:i+1]
            y_sub = y[prev:i+1]
            z_sub = z[prev:i+1]
            #print("time_sub:",time_sub)
            #print("x_sub:",x_sub)
            #print("y_sub:",y_sub)
            #print("z_sub:",z_sub)
            
            
            features =  computeFeatures(time_prev, time[i], time_sub, x_sub, y_sub, z_sub)
           # print(features)
            
            f_all.append([features])
            print("features:", f_all )
            
            time_prev = time[i+1]
            prev = i+1
            #print(time_prev)
            
    
    

#     print(D.head())


Mean: 1043.25647853
Meadian: 1029.82032574
Std: 165.276900025
Std_x 290.77942005529263
Std_y 192.7942452284006
Std_z 246.86004424215537
Skewness: [ 0.63743938]
Kurtosis: [ 1.17167717]
roc: [ 0.04478513]
features: [[[80.027578897503503, 10091.893100184201, 1043.2564785318309, 1029.8203257362575, 165.27690002463947, 290.77942005529263, 192.7942452284006, 246.86004424215537, array([ 0.63743938]), array([ 1.17167717]), array([ 0.04478513])]]]
Mean: 1023.00483177
Meadian: 1022.99991651
Std: 10.5742416064
Std_x 13.008628927128415
Std_y 13.192250528858837
Std_z 37.669829284571236
Skewness: [-0.00368798]
Kurtosis: [ 0.40933366]
roc: [ 0.00383555]
features: [[[80.027578897503503, 10091.893100184201, 1043.2564785318309, 1029.8203257362575, 165.27690002463947, 290.77942005529263, 192.7942452284006, 246.86004424215537, array([ 0.63743938]), array([ 1.17167717]), array([ 0.04478513])]], [[10154.2414104371, 20171.910250978799, 1023.0048317692793, 1022.999916514073, 10.574241606406515, 13.00862892712

In [5]:
import numpy as np, pandas as pd
import math
from scipy.stats import skew
from scipy.stats import kurtosis

def getRateOfChange(timestamp, magnitude):
    roc = 0;    
    for i in range(len(magnitude)-1):                
        roc = roc + (((magnitude[i+1] - magnitude[i]) / (timestamp.iloc[i+1] - timestamp.iloc[i])));
    roc = roc / (len(magnitude)- 1);
    
    return roc

def computeFeatures(ts, x, y, z):
    mag = np.empty([len(x), 1])
    for i in range(len(x)):
        mag[i] = math.sqrt(x.iloc[i]*x.iloc[i] + y.iloc[i]*y.iloc[i]+z.iloc[i]*z.iloc[i])

    #a = [2, 8, 0, 4, 1, 9, 9, 0];
    mean =  np.mean(mag);
    median = np.median(mag);
    std = np.std(mag);
    std_x = np.std(x);
    std_y = np.std(y);
    std_z = np.std(z);
    skewness = skew(mag);
    kurt = kurtosis(mag);
    rateOfChanges=getRateOfChange(ts, mag)
    
    f = [mean, median, std, std_x, std_y, std_z, skewness, kurt, rateOfChanges]
    return f

def computeWindowFeatures(ts, x, y, z, window_size):
    time_prev = ts[0]
    prev = 0
    features_all = []
    for i in range(len(ts)):
        if (ts[i] - time_prev) >= window_size:
            time_sub = ts[prev:i+1]
            x_sub = x[prev:i+1]
            y_sub = y[prev:i+1]
            z_sub = z[prev:i+1]
            features =  computeFeatures(time_sub, x_sub, y_sub, z_sub)
            
            features_all.append(features)
            time_prev = ts[i+1]
            prev = i+1
    return features_all

if __name__ == "__main__":
    column_names = ['timestamp', 'A_x', 'A_y', 'A_z']
    D = pd.read_csv('C:/Research/Data/Activity/right-wrist-accelxyz.csv', names = column_names)
    
    x= D['A_x']
    y= D['A_y']
    z= D['A_z']
    ts= D['timestamp']
    
    features = computeWindowFeatures(ts, x, y, z, window_size=10*1000)
    
    print(len(features))
  



2


In [28]:
import os
import numpy as np, pandas as pd
import math
from scipy.stats import skew
from scipy.stats import kurtosis

def getRateOfChange(timestamp, magnitude):
    roc = 0;    
    for i in range(len(magnitude)-1):                
        roc = roc + (((magnitude[i+1] - magnitude[i]) / (timestamp.iloc[i+1] - timestamp.iloc[i])));
    roc = roc / (len(magnitude)- 1);
    
    return roc

def computeFeatures(ts, x, y, z):
    mag = np.empty([len(x), 1])
    for i in range(len(x)):
        mag[i] = math.sqrt(x.iloc[i]*x.iloc[i] + y.iloc[i]*y.iloc[i]+z.iloc[i]*z.iloc[i])

    #a = [2, 8, 0, 4, 1, 9, 9, 0];
    mean =  np.mean(mag);
    median = np.median(mag);
    std = np.std(mag);
    std_x = np.std(x);
    std_y = np.std(y);
    std_z = np.std(z);
    skewness = skew(mag);
    kurt = kurtosis(mag);
    rateOfChanges=getRateOfChange(ts, mag)
    
    f = [mean, median, std, std_x, std_y, std_z, skewness[0], kurt[0], rateOfChanges[0]]
    return f

def computeWindowFeatures(ts, x, y, z, window_size):
    time_prev = ts[0]
    prev = 0
    features_all = []
    for i in range(len(ts)):
        if (ts[i] - time_prev) >= window_size:
            time_sub = ts[prev:i+1]
            x_sub = x[prev:i+1]
            y_sub = y[prev:i+1]
            z_sub = z[prev:i+1]
            features =  computeFeatures(time_sub, x_sub, y_sub, z_sub)
            
            features_all.append(features)
            time_prev = ts[i+1]
            prev = i+1
    return features_all


def getActivityLevelmap():
    NO_ACT = 'NO_ACT';
    LOW = 'LOW';
    MOD = 'MODERATE';
    HIGH = 'HIGH';
    
    M = dict()

    M['LYING DOWN'] = NO_ACT
    M['SLEEPING'] = NO_ACT
    M['CLEANING'] = LOW
    M['COMPUTER WORK'] = LOW
    M['COOKING'] = LOW
    M['DOING LAUNDRY'] = LOW
    M['EATING'] = LOW
    M['LAB WORK'] = LOW
    M['SHOPPING'] = LOW
    M['WASHING DISHES'] = LOW
    M['WATCHING TV'] = LOW
    M['STAIRS - GOING DOWN'] = MOD
    M['STAIRS - GOING UP'] = MOD
    M['STROLLING'] = MOD
    M['walking'] = MOD
    M['exercise'] = HIGH
    M['running'] = HIGH
    M['BICYCLING'] = HIGH
    
    return M


# [x[0] for x in os.walk(directory)]
if __name__ == "__main__":
    dirName = 'C:/Research/Data/Activity/streamprocessor-like-data-25hz/'
    filename = 'right-wrist-accelxyz.csv'
    column_names = ['timestamp', 'A_x', 'A_y', 'A_z']

    M = getActivityLevelmap()
    
    subDirs = os.listdir(dirName)
    
    all_features = []
    actual_label = []
    activity_level = []
    cnt = 0
    for subDir in subDirs:
#         if cnt>10:
#             break
        print(subDir)
        pidsids = os.listdir(dirName + subDir )
        for pidsid in pidsids:
#             if cnt>10:
#                 break
            cnt=cnt+1
            D = pd.read_csv(dirName + subDir + '/' + pidsid + '/' + filename, names = column_names)
            x= D['A_x']
            y= D['A_y']
            z= D['A_z']
            ts= D['timestamp']
            features = computeWindowFeatures(ts, x, y, z, window_size=10*1000)

            for f in features:
                all_features.append(f)
                actual_label.append([subDir])
                activity_level.append([M[subDir]])
    
    print(len(all_features))
    print(len(actual_label))
    print(len(activity_level))
    a = np.array(all_features)
    b = np.array(actual_label)
    c = np.array(activity_level)
#     print(a)
#     print(b)
#     print(c)
    
    D = np.concatenate( (a, b, c), axis=1)
#     print(D)

#     foo = np.array([1,2,3])
#     with open(dirName + 'featureFile.csv', 'wb') as f:
#         np.savetxt(f, D, delimiter=",")
    np.savetxt(dirName + 'featureFile.csv', D, delimiter=',', fmt="%s")

BICYCLING
CLEANING
COMPUTER WORK


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


COOKING
DOING LAUNDRY
EATING
exercise
LAB WORK
LYING DOWN
running
SHOPPING
SLEEPING
STAIRS - GOING DOWN
STAIRS - GOING UP
STROLLING
walking
WASHING DISHES
WATCHING TV
160947
160947
160947


In [22]:
dirName = 'C:/Research/Data/Activity/streamprocessor-like-data-25hz/'
print(os.listdir(dirName))

['BICYCLING', 'CLEANING', 'COMPUTER WORK', 'COOKING', 'DOING LAUNDRY', 'EATING', 'exercise', 'LAB WORK', 'LYING DOWN']


In [3]:
import numpy as np, pandas as pd

A=[[1, 2, 3],
   [2,3, 4],
   [4, 5, 6]]
B = [['a'], ['b'], ['c']]
C = [['aa'], ['ab'], ['ac']]

print(A)
print(B)
a = np.array(A)
b = np.array(B)
c = np.array(C)
print(np.concatenate( (a, b, c), axis=1))

[[1, 2, 3], [2, 3, 4], [4, 5, 6]]
[['a'], ['b'], ['c']]
[['1' '2' '3' 'a' 'aa']
 ['2' '3' '4' 'b' 'ab']
 ['4' '5' '6' 'c' 'ac']]


In [38]:
A = []
A.append([1, 2, 3])
A.append([1, 2, 3])
print(A)

[[1, 2, 3], [1, 2, 3]]
